In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install openslide-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openslide-python: filename=openslide_python-1.3.1-cp310-cp310-linux_x86_64.whl size=33544 sha256=d1ab28cb27812e577409ce9338a271786ff6a45149a6bfcafbe1ece2ac3320fe
  Stored in directory: /root/.cache/pip/wheels/79/79/fa/29a0087493c69dff7fd0b70fab5d6771002a531010161d2d97
Successfully built openslide-python


In [4]:
!apt-get install -y openslide-tools


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 45 not upgraded.
Need to get 104 kB of archives.
After this operation, 297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenslide0 amd64 3.4.1+dfsg-5build1 [89.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 openslide-tools amd64 3.4.1+dfsg-5build1 [13.8 kB]
Fetched 104 kB in 1s (119 kB/s)
Selecting previously unselected package libopenslide0.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../libopenslide0_3.4.1+dfsg-5build1_amd64.deb ...
Unpacking libopenslide0 (3.4.1+dfsg-5build1) ...
Selecting previously unselected package openslide-tools.

In [2]:
import openslide
import os
from PIL import Image

def load_images_from_folder(folder, label, resize_width=256, resize_height=256):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".svs"):
            path = os.path.join(folder, filename)
            slide = openslide.OpenSlide(path)
            level_count = slide.level_count
            level_dimensions = slide.level_dimensions
            level = 0  # Assuming level 0 has the lowest resolution
            width, height = level_dimensions[level]

            # Starting dimensions for cropping
            start_width = 0
            start_height = 0
            cropped_width = width // 5
            cropped_height = height // 5

            # Loop through 5 rows and columns to create 25 cropped images
            for row in range(5):
                for col in range(5):
                    # Load a region of the slide
                    region = slide.read_region(location=(start_width, start_height), level=level, size=(cropped_width, cropped_height))
                    # Resize the image
                    region = region.resize((resize_width, resize_height))
                    image = region.convert("RGB")

                    images.append(image)
                    labels.append(label)

                    # Update starting dimensions for next cropped image
                    start_width += cropped_width
                start_height += cropped_height
                start_width = 0  # Reset start width for next row

            # Close the slide
            slide.close()

    return images, labels

# Load LGG and GBM images with labels
train_lgg_images, train_lgg_labels = load_images_from_folder("/content/drive/MyDrive/pe_ramesh_sir/lgg", label="lgg")
train_gbm_images, train_gbm_labels = load_images_from_folder("/content/drive/MyDrive/pe_ramesh_sir/gbm", label="gbm")
test_images, _ = load_images_from_folder("/content/drive/MyDrive/pe_ramesh_sir/test", label=None)

# You can adjust the resize_width and resize_height parameters as needed


ModuleNotFoundError: No module named 'openslide'

In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import multiprocessing

# Load CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Move model to appropriate device
clip_model.to(device)

def process_image(image, label):
    image_features = []
    inputs = clip_processor(text=label, images=image, return_tensors="pt")
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Move inputs to device
    with torch.no_grad():
        image_feature = clip_model.get_image_features(pixel_values=inputs["pixel_values"])
    image_features.append(image_feature.cpu().numpy())
    return np.concatenate(image_features, axis=0)

def extract_features(images, labels, batch_size=1):
    features = []

    pool = multiprocessing.Pool()
    results = pool.starmap(process_image, zip(images, labels))
    pool.close()
    pool.join()

    return np.array(results)

# Extract features for training and testing images
train_lgg_features = extract_features(train_lgg_images, train_lgg_labels)
train_gbm_features = extract_features(train_gbm_images, train_gbm_labels)
test_features = extract_features(test_images, [None]*len(test_images))



In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Combine features and labels
X_train_flat = np.concatenate((train_lgg_features, train_gbm_features), axis=0)
# Flatten the second dimension
X_train_flat = X_train_flat.reshape(X_train_flat.shape[0], -1)
y_train = np.concatenate((train_lgg_labels, train_gbm_labels))


print("Shape of X_train_flat:", X_train_flat.shape)


# Train logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train_flat, y_train)

# Make predictions based on majority voting
def predict_majority(features):
    num_images = len(features)
    num_cropped_images = features.shape[0] // num_images  # Calculate based on the shape of features
    print("Number of cropped images per original image:", num_cropped_images)

    # Reshape features to (num_images * num_cropped_images, feature_dim)
    features_reshaped = features.reshape(-1, features.shape[-1])

    final_predictions = []

    for i in range(0, len(features_reshaped), num_cropped_images):
        # Predict labels for each cropped image of the current original image
        predictions = classifier.predict(features_reshaped[i:i+num_cropped_images])

        # Determine majority vote for the current original image
        majority_votes = np.sum(predictions == "gbm") > (num_cropped_images // 2)

        # Convert majority vote to label for the current original image
        final_prediction = "gbm" if majority_votes else "lgg"

        # Append final prediction for the current original image
        final_predictions.append(final_prediction)

    return np.array(final_predictions)



# Evaluate performance
def evaluate(predictions, true_labels):
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

# Evaluate on training set
train_predictions = predict_majority(np.concatenate((train_lgg_features, train_gbm_features), axis=0))
train_accuracy = evaluate(train_predictions, y_train)
print("Training Accuracy:", train_accuracy)

In [ ]:
# Make predictions on test set using majority voting
test_predictions = predict_majority(test_features)

# Display predictions
for i, prediction in enumerate(test_predictions):
    if prediction == "lgg":
        print(f"Test image {i+1} is classified as LGG.")
    else:
        print(f"Test image {i+1} is classified as GBM.")


In [ ]:
# # Obtain true labels for the test set
# # Assuming you have the true labels stored in a variable called test_labels
# # Replace 'test_labels' with the actual variable name if it's different
# test_labels = ['lgg', 'lgg','gbm','gbm']  # Example of test labels, replace this with your actual test labels

# # Make predictions on test set
# test_predictions = predict(np.concatenate(test_features))

# # Calculate testing accuracy
# test_accuracy = evaluate(test_predictions, test_labels)
# print("Testing Accuracy:", test_accuracy)


In [ ]:
# Obtain true labels for the test set
# Assuming you have the true labels stored in a variable called test_labels
# Replace 'test_labels' with the actual variable name if it's different
test_labels = ['lgg', 'lgg', 'gbm', 'gbm']  # Example of test labels, replace this with your actual test labels

# Make predictions on test set using majority voting
test_predictions = predict_majority(test_features)

# Calculate testing accuracy
test_accuracy = evaluate(test_predictions, test_labels)
print("Testing Accuracy:", test_accuracy)

In [ ]:
# Flatten the feature vectors
test_features_flat = np.vstack(test_features)

# Make predictions on the test set
test_predictions = classifier.predict(test_features_flat)

# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print("Test Accuracy:", test_accuracy)

# Calculate precision, recall, and F1-score
precision_test = precision_score(test_labels, test_predictions, average='weighted')
recall_test = recall_score(test_labels, test_predictions, average='weighted')
f1_test = f1_score(test_labels, test_predictions, average='weighted')
print("Precision (Test):", precision_test)
print("Recall (Test):", recall_test)
print("F1-score (Test):", f1_test)

# Compute confusion matrix
conf_matrix_test = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix (Test):")
print(conf_matrix_test)


In [ ]:
for i, (true_label, pred_label) in enumerate(zip(test_labels, test_predictions)):
    print(f"Sample {i+1}: True label - {true_label}, Predicted label - {pred_label}")

# PCA


In [ ]:
# Flatten the feature vectors
all_features_flat = all_features.reshape(all_features.shape[0], -1)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=2)
pca_features = pca.fit_transform(all_features_flat)

# Plot the PCA features
plt.figure(figsize=(10, 6))
for label in np.unique(all_labels):
    plt.scatter(pca_features[all_labels == label, 0], pca_features[all_labels == label, 1], label=label)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Visualization of CLIP Features')
plt.legend()
plt.show()


#  cross-validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

# Define the number of folds for cross-validation
n_splits = 5  # You can adjust this as needed

# Define cross-validation strategy
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize lists to store accuracy scores for each fold
cv_train_accuracies = []
cv_val_accuracies = []

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(skf.split(X_train_flat, y_train), 1):
    print(f"Fold {fold}/{n_splits}:")
    # Split data into training and validation sets
    X_train_fold, X_val_fold = X_train_flat[train_index], X_train_flat[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # Train logistic regression classifier
    classifier = LogisticRegression()
    classifier.fit(X_train_fold, y_train_fold)

    # Make predictions on training and validation sets
    train_predictions = classifier.predict(X_train_fold)
    val_predictions = classifier.predict(X_val_fold)

    # Calculate accuracy scores
    train_accuracy = accuracy_score(y_train_fold, train_predictions)
    val_accuracy = accuracy_score(y_val_fold, val_predictions)

    print(f"Training Accuracy: {train_accuracy}")
    print(f"Validation Accuracy: {val_accuracy}\n")

    # Store accuracy scores
    cv_train_accuracies.append(train_accuracy)
    cv_val_accuracies.append(val_accuracy)

# Calculate average accuracy across all folds
avg_train_accuracy = np.mean(cv_train_accuracies)
avg_val_accuracy = np.mean(cv_val_accuracies)

print(f"Average Training Accuracy across all folds: {avg_train_accuracy}")
print(f"Average Validation Accuracy across all folds: {avg_val_accuracy}")


Data Augmentation: Apply data augmentation techniques such as rotation, flipping, scaling, or adding noise to increase the diversity of your training data.


In [ ]:
import openslide
import os
import openslide
from PIL import Image
import torch
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define data augmentation transforms
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".svs"):
            path = os.path.join(folder, filename)
            slide = openslide.OpenSlide(path)
            level_count = slide.level_count
            level_dimensions = slide.level_dimensions
            level = 0  # Assuming level 0 has the lowest resolution
            width, height = level_dimensions[level]

            # Load a region of the slide (e.g., top-left corner)
            region = slide.read_region(location=(0, 0), level=level, size=(width // 10, height // 10))
            image = region.convert("RGB")

            # Apply data augmentation
            image = data_transforms(image)

            images.append(image)
            labels.append(label)
    return images, labels

# Load LGG and GBM images with labels
train_lgg_images, train_lgg_labels = load_images_from_folder("/kaggle/input/training-lgg/lgg", label="lgg")
train_gbm_images, train_gbm_labels = load_images_from_folder("/kaggle/input/gbm-training-dataset/gbm", label="gbm")
test_images, _ = load_images_from_folder("/kaggle/input/test-dataset-tumor/test_folder", label=None)


In [ ]:

# Load CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Move model to appropriate device
clip_model.to(device)

# Extract features for training images
def extract_features(images, labels):
    features = []
    for image, label in zip(images, labels):
        inputs = clip_processor(text=label, images=image.unsqueeze(0), return_tensors="pt")
        inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Move inputs to device
        with torch.no_grad():
            image_features = clip_model.get_image_features(pixel_values=inputs["pixel_values"])
        features.append(image_features.cpu().numpy())
    return features


# Extract features for training and testing images
train_lgg_features = extract_features(train_lgg_images, train_lgg_labels)
train_gbm_features = extract_features(train_gbm_images, train_gbm_labels)
test_features = extract_features(test_images, [None]*len(test_images))  # No labels for test set

# Combine features and labels
X_train_flat = np.concatenate((train_lgg_features, train_gbm_features)).reshape(-1, train_lgg_features[0].shape[-1])
y_train = np.concatenate((train_lgg_labels, train_gbm_labels))

# Train logistic regression classifier
classifier = LogisticRegression()
classifier.fit(X_train_flat, y_train)

# Make predictions
def predict(features):
    features_flat = features.reshape(features.shape[0], -1)
    predictions = classifier.predict(features_flat)
    return predictions

# Evaluate performance
def evaluate(predictions, true_labels):
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

# Evaluate on training set
train_predictions = predict(np.concatenate((train_lgg_features, train_gbm_features)))
train_accuracy = evaluate(train_predictions, y_train)
print("Training Accuracy:", train_accuracy)

# Make predictions on test set
test_predictions = predict(np.concatenate(test_features))

# Display predictions
for i, prediction in enumerate(test_predictions):
    if prediction == "lgg":
        print(f"Test image {i+1} is classified as LGG.")
    else:
        print(f"Test image {i+1} is classified as GBM.")


with data augumentation and using svm and random forest for classifying

In [ ]:
import openslide
import os
import openslide
from PIL import Image
import torch
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define data augmentation transforms
data_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor()
])

def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith(".svs"):
            path = os.path.join(folder, filename)
            slide = openslide.OpenSlide(path)
            level_count = slide.level_count
            level_dimensions = slide.level_dimensions
            level = 0  # Assuming level 0 has the lowest resolution
            width, height = level_dimensions[level]

            # Load a region of the slide (e.g., top-left corner)
            region = slide.read_region(location=(0, 0), level=level, size=(width // 10, height // 10))
            image = region.convert("RGB")

            # Apply data augmentation
            image = data_transforms(image)

            images.append(image)
            labels.append(label)
    return images, labels

# Load LGG and GBM images with labels
train_lgg_images, train_lgg_labels = load_images_from_folder("/kaggle/input/training-lgg/lgg", label="lgg")
train_gbm_images, train_gbm_labels = load_images_from_folder("/kaggle/input/gbm-training-dataset/gbm", label="gbm")
test_images, _ = load_images_from_folder("/kaggle/input/test-dataset-tumor/test_folder", label=None)

# Load CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Move model to appropriate device
clip_model.to(device)

# Extract features for training images
def extract_features(images, labels):
    features = []
    for image, label in zip(images, labels):
        inputs = clip_processor(text=label, images=image.unsqueeze(0), return_tensors="pt")
        inputs = {key: tensor.to(device) for key, tensor in inputs.items()}  # Move inputs to device
        with torch.no_grad():
            image_features = clip_model.get_image_features(pixel_values=inputs["pixel_values"])
        features.append(image_features.cpu().numpy())
    return features


# Extract features for training and testing images
train_lgg_features = extract_features(train_lgg_images, train_lgg_labels)
train_gbm_features = extract_features(train_gbm_images, train_gbm_labels)
test_features = extract_features(test_images, [None]*len(test_images))  # No labels for test set

# Combine features and labels
X_train_flat = np.concatenate((train_lgg_features, train_gbm_features)).reshape(-1, train_lgg_features[0].shape[-1])
y_train = np.concatenate((train_lgg_labels, train_gbm_labels))

# Train SVM classifier
svm_classifier = SVC()
svm_classifier.fit(X_train_flat, y_train)

# Train Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_flat, y_train)

# Make predictions
def predict(classifier, features):
    features_flat = features.reshape(features.shape[0], -1)
    predictions = classifier.predict(features_flat)
    return predictions

# Evaluate performance
def evaluate(predictions, true_labels):
    accuracy = accuracy_score(true_labels, predictions)
    return accuracy

# Evaluate SVM on training set
train_svm_predictions = predict(svm_classifier, np.concatenate((train_lgg_features, train_gbm_features)))
train_svm_accuracy = evaluate(train_svm_predictions, y_train)
print("SVM Training Accuracy:", train_svm_accuracy)

# Evaluate Random Forest on training set
train_rf_predictions = predict(rf_classifier, np.concatenate((train_lgg_features, train_gbm_features)))
train_rf_accuracy = evaluate(train_rf_predictions, y_train)
print("Random Forest Training Accuracy:", train_rf_accuracy)

# Make predictions on test set using SVM
test_svm_predictions = predict(svm_classifier, np.concatenate(test_features))

# Display predictions
for i, prediction in enumerate(test_svm_predictions):
    if prediction == "lgg":
        print(f"Test image {i+1} is classified as LGG (SVM).")
    else:
        print(f"Test image {i+1} is classified as GBM (SVM).")

# Make predictions on test set using Random Forest
test_rf_predictions = predict(rf_classifier, np.concatenate(test_features))

# Display predictions
for i, prediction in enumerate(test_rf_predictions):
    if prediction == "lgg":
        print(f"Test image {i+1} is classified as LGG (Random Forest).")
    else:
        print(f"Test image {i+1} is classified as GBM (Random Forest).")
